# Plotting pre-processed SST1 RSoXS data
Use pyhyper's new RSoXS plotting shortctus

## Imports

In [ ]:
## Imports
import PyHyperScattering as phs
import pathlib
import sys
import io
import ast
import json
import datetime
import dask.array as da
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from tqdm.auto import tqdm
import subprocess
import gc

print(f'Using PyHyperScattering Version: {phs.__version__}')

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

## Define paths

In [ ]:
basePath = pathlib.Path.cwd()

## Load cartesian data 

In [ ]:
[f.name for f in basePath.glob('*zarr')]

In [ ]:
DS = xr.open_zarr(basePath.joinpath('cartesian_data_PM6_CF.zarr'))

# Compute any dask coordiantes
for coord_name, coord_data in DS.coords.items():
    if isinstance(coord_data.data, da.Array):
        DS.coords[coord_name] = coord_data.compute()
        
DS

### Detector images

#### Facet plot at several selected energies

#### Single image at selected energy

#### Generate mp4 movie of detector image at all energies

## Load polar data

In [ ]:
[f.name for f in basePath.glob('*zarr')]

In [ ]:
DS = xr.open_zarr(basePath.joinpath('polar_data_PM6_CF.zarr'))

# Compute any dask coordiantes
for coord_name, coord_data in DS.coords.items():
    if isinstance(coord_data.data, da.Array):
        DS.coords[coord_name] = coord_data.compute()
        
DS

### Integrated scattered intensity (ISI) plots

In [ ]:
# Select a dataarray (potentially from larger dataset)
pol = 0
data_variable = 'raw_intensity'
DA = DS[data_variable].sel(polarization=pol)

# Set parameters for ISI function
chi_width = 90  # width of chi wedge for para / perp slices
q_slice = slice(0.01, 0.09)  # q range (as slice object)
e_slice = slice(280, 295)  # energy range (as slice object)
sample_name = str(DA.sample_name.values)  # set custom sample name, ideally from DataArray 

# Call ISI function, return ISI figure & axes object
fig, ax = DA.pt.plot_ISI(pol, chi_width, q_slice, e_slice, sample_name)

# Save if desired
save = False
filename = f'{sample_name}_chi-{chi_width}_q-{q_slice.start}-{q_slice.stop}_energy-{e_slice.start}-{e_slice.stop}_.png'
if save:
    savePath = basePath.joinpath('ISI_plots')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath(f'{sample_name}.png'))

# Plot
plt.show()
plt.close('all')

In [ ]:
# quick plotting to find q limits

# %matplotlib widget
# plt.close('all')
# sel_DA = DA.sel(energy=285)
# cmin, cmax = sel_DA.quantile([0.01, 0.99]).compute()
# sel_DA.plot(norm=LogNorm(cmin,cmax), cmap=plt.cm.turbo)
# plt.show()

%matplotlib widget
plt.close('all')
sel_DA = perp_DA.sel(energy=285, method='nearest').interpolate_na(dim='q')
cmin, cmax = sel_DA.quantile([0.01,0.99]).compute()
sel_DA.plot(norm=LogNorm(cmin,cmax))
plt.show()

### Intensity heatmaps

### Intensity vs Q linecuts

### Anisotropy heatmaps

### Anisotropy vs Q linecuts

## Previous, less organized code below:

In [ ]:
# misc helper function
def make_para_perp_DAs(DS, sample_name, intensity_type, pol, qlims, chi_width):
    # select dataarray to plot
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    sliced_DA = DA.sel(polarization=pol, q=slice(qlims[0],qlims[1]))

    # calculate ISI dataarrays
    if pol==0:
        para_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        perp_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))
    elif pol==45:
        para_DA = sliced_DA.rsoxs.slice_chi(-135, chi_width=(chi_width/2))
        perp_DA = sliced_DA.rsoxs.slice_chi(135, chi_width=(chi_width/2))        
    elif pol==90:
        perp_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        para_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))   
        
    return para_DA, perp_DA

## Intensity Plots

### 1. Detector movies

In [ ]:
%matplotlib inline

In [ ]:
plt.close('all')

In [ ]:
already_made_movies = set([f.name[:-17] for f in plotsPath.joinpath('rsoxs_carbon_detector_movies_v1').glob('*pol90*')])

In [ ]:
selected_samples = list(set(DS.sample_name.values).difference(already_made_movies))
selected_samples

In [ ]:
# Parameters
edge = 'carbon'
# DS = rsoxs_datasets[f'cartesian_{edge}']
# DS = rsoxs_datasets[f'cartesian_{edge}_reverse']
intensity_type = 'raw'
pol = 0
energy = 285.2

# selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
#                     'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
#                     'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 
# selected_samples = ['PM6_5CN-CB']

# for sample_name in DS.sample_name.values:
for sample_name in selected_samples:
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    DA = DA.where(DA>0, 10)
    # Make & customize plot
    sliced_DA = DA.sel(polarization=pol).sel(energy=energy, method='nearest').swap_dims({'pix_x':'qx','pix_y':'qy'})

    cmin = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.03))
    cmax = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.999))
    
    # cmin = cmin if cmin > 1e8 else 1e8

    ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=LogNorm(cmin,cmax))
    ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
    ax.figure.set_tight_layout(True)   
    ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
    ax.colorbar.set_label('Raw Intensity [arb. units]', rotation=270, labelpad=12)

    plt.show()
    plt.close('all')

In [ ]:
# Select Dataset
# edge = 'carbon'
# DS = rsoxs_datasets[f'cartesian_{edge}_reverse']
# DS = rsoxs_datasets[f'cartesian_{edge}_tilted']

# bcx = DS['raw_intensity'].beamcenter_x
# bcy = DS['raw_intensity'].beamcenter_y

# Select Plotting Parameters
# pol = 90
# pix_size = 500
# pix_x_slice = slice(bcx-(pix_size/2), bcx+(pix_size/2))
# pix_y_slice = slice(bcy-(pix_size/2), bcy+(pix_size/2))
intensity_type = 'raw'

# Select DataArray
# sample_name = 'PM6-Y6_3000_dSiN'
# selected_samples = ['PM6_5CN-CB']
for pol in [0, 90]:
    # for sample_name in tqdm(DS.sample_name.values, desc=f'Polarization {pol}°'):
    for sample_name in tqdm(selected_samples, desc=f'Polarization {pol}°'):
        DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']

        savePath = plotsPath.joinpath('rsoxs_carbon_detector_movies_v1')
        savePath.mkdir(exist_ok=True)
        output_path = savePath.joinpath(f'{sample_name}_{intensity_type}_pol{pol}deg.mp4')

        # FFmpeg command. This is set up to accept data from the pipe and use it as input, with PNG format.
        # It will then output an H.264 encoded MP4 video.
        cmd = [
            'ffmpeg',
            '-y',  # Overwrite output file if it exists
            '-f', 'image2pipe',
            '-vcodec', 'png',
            '-r', '16',  # Frame rate
            '-i', '-',  # The input comes from a pipe
            '-vcodec', 'libx264',
            '-pix_fmt', 'yuv420p',
            '-crf', '22',  # Set the quality (lower is better, 17 is often considered visually lossless)
            str(output_path)
        ]

        # Start the subprocess
        proc = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Loop through the energy dimension and send frames to FFmpeg
        for i, energy in enumerate(tqdm(DA.energy.values, desc=f'Making the {sample_name} movie')):
            # Make & customize plot
            sliced_DA = DA.sel(polarization=pol).sel(energy=energy, method='nearest').swap_dims({'pix_x':'qx','pix_y':'qy'})
            
#             cmin = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.05))
#             cmax = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.999))

#             cmin = cmin if cmin > 1e8 else 1e8
            
            ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=LogNorm(cmin,cmax))
            ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
            ax.figure.set_tight_layout(True)   
            ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
            ax.colorbar.set_label('Raw Intensity [arb. units]', rotation=270, labelpad=12)

            # Save first frame (or specific energy value)
            if energy == 285.2:
                ax.figure.savefig(savePath.joinpath(f'{sample_name}_{intensity_type}_pol{pol}deg.png'), dpi=120)

            buf = io.BytesIO()
            ax.figure.savefig(buf, format='png')
            buf.seek(0)

            # Write the PNG buffer data to the process
            proc.stdin.write(buf.getvalue())
            plt.close('all')

        # Finish the subprocess
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(f"Error: {err}")
            
        gc.collect()  # try to clear up some memory...


In [ ]:
plotsPath

### 2. ISI Plots

In [ ]:
# Select polar dataset, to ISI plots, polarizations 0 & 90 averaged
edge = 'carbon'
DS = rsoxs_datasets[f'polar_{edge}_reverse'].copy()
# DS = rsoxs_datasets[f'polar_{edge}_tilted'].copy()

# DS = DS.rename_vars({'raw_intensity':'corr_intensity'})

chi_width = 90
q_slice = slice(0.009,0.08)
e_slice = slice(282, 295)

# make selection
intensity_type = 'corr'

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
#     # Fl correction?
#     scale_factor = float(para_DA.sel(energy=slice(285,305), q=slice(2e-2,6e-2)).mean('chi').mean('energy').integrate('q'))
#     manual_scale_factor = manual_scale_factors_v4[sample_name]
#     trmsn90_corr = (manual_scale_factor * scale_factor * trmsn90_bkgs_DA.sel(sample_name=sample_name))
    
#     para_DA = para_DA - trmsn90_corr
#     perp_DA = perp_DA - trmsn90_corr

    # slice ISI data
    para_ISI = para_DA.interpolate_na(dim='q').sel(q=q_slice).mean('chi').mean('q')
    perp_ISI = perp_DA.interpolate_na(dim='q').sel(q=q_slice).mean('chi').mean('q')

    # plot
    fig, ax = plt.subplots(figsize=(6,4), dpi=120)
    ax.xaxis.set_minor_locator(MultipleLocator(1))

    para_ISI.sel(energy=e_slice).plot.line(ax=ax, label='$\parallel$', yscale='log')
    perp_ISI.sel(energy=e_slice).plot.line(ax=ax, label='$\perp$', yscale='log')
    fig.suptitle(f'ISI: {sample_name}', fontsize=14, x=0.55)
    ax.set(title=f'Pol = {pol}°, chi width = {chi_width}°, Q = ({q_slice.start}, {q_slice.stop}) ' + 'Å$^{-1}$', 
           xlabel='X-ray energy [eV]', ylabel='Intensity [arb. units]')
    ax.legend(loc='upper right', fontsize=14)
    ax.grid(axis='x', which='both')
    plt.subplots_adjust(top=0.86, bottom=0.2, left=0.2)
        
    # savePath = plotsPath.joinpath('rsoxs_carbon_reverse/ISIs_v1')
    # savePath.mkdir(exist_ok=True)
    # fig.savefig(savePath.joinpath( 
    #     f'{sample_name}_{intensity_type}_chi{chi_width}deg_q{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

    # plt.show()
    plt.close('all')

In [ ]:
plotsPath

In [ ]:
# Select polar dataset, to ISI plots, individual polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}'].copy()
DS = rsoxs_datasets[f'polar_{edge}_tilted'].copy()

# DS = DS.rename_vars({'raw_intensity':'corr_intensity'})

chi_width = 90
q_slice = slice(0.009,0.08)
e_slice = slice(282, 295)

# make selection
intensity_type = 'corr'

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 

        # slice ISI data
        para_ISI = para_DA.interpolate_na(dim='q').sel(q=q_slice).mean('chi').mean('q')
        perp_ISI = perp_DA.interpolate_na(dim='q').sel(q=q_slice).mean('chi').mean('q')

        # plot
        fig, ax = plt.subplots(figsize=(6,4), dpi=120)
        ax.xaxis.set_minor_locator(MultipleLocator(1))

        para_ISI.sel(energy=e_slice).plot.line(ax=ax, label='$\parallel$', yscale='log')
        perp_ISI.sel(energy=e_slice).plot.line(ax=ax, label='$\perp$', yscale='log')
        fig.suptitle(f'ISI: {sample_name}', fontsize=14, x=0.55)
        ax.set(title=f'Pol = {pol}°, chi width = {chi_width}°, Q = ({q_slice.start}, {q_slice.stop}) ' + 'Å$^{-1}$', 
               xlabel='X-ray energy [eV]', ylabel='Intensity [arb. units]')
        ax.legend(loc='upper right', fontsize=14)
        ax.grid(axis='x', which='both')
        plt.subplots_adjust(top=0.86, bottom=0.2, left=0.2)

        savePath = plotsPath.joinpath('rsoxs_carbon_tilted/ISIs_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath( 
            f'{sample_name}_{intensity_type}_chi{chi_width}deg_q{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

        # plt.show()
        plt.close('all')

In [ ]:
selected_samples = ['Y6_3000_dSiN', 'PM6_3000_dSiN', 'PM6-Y6_3000_dSiN', 'PM7-Y6_3000_dSiN', 'Y7_3000_dSiN', 'PM7_3000_dSiN', 'PM7-Y7_3000_dSiN']
# selected_samples = ['Y6_3000_dSiN', 'PM6_3000_dSiN', 'PM6-Y6_3000_dSiN', 'PM7-Y6_3000_dSiN', 'Y7_3000_dSiN', 'PM7_3000_dSiN', 'PM7-Y7_3000_dSiN']

### 3. 2D Linecut Maps

In [ ]:
# Select polar dataset, to I cuts, averaged 0 & 90 deg polarization
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}'].copy()
# DS = rsoxs_datasets[f'polar_{edge}_tilted'].copy()
DS = rsoxs_datasets[f'polar_{edge}_reverse'].copy()

chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

# make selection
# sample_name = 'PM6-Y6_3000_dSiN'
intensity_type = 'corr'
pol = 0

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    # for pol in [0]:
    #     para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
   
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
#     # Fl correction?
#     scale_factor = float(para_DA.sel(energy=slice(285,305), q=slice(2e-2,6e-2)).mean('chi').mean('energy').integrate('q'))
#     manual_scale_factor = manual_scale_factors_v4[sample_name]
#     trmsn90_corr = (manual_scale_factor * scale_factor * trmsn90_bkgs_DA.sel(sample_name=sample_name))
    
#     para_DA = para_DA - trmsn90_corr
#     perp_DA = perp_DA - trmsn90_corr
                
    # Plot
    fig, axs = plt.subplots(1, 2, figsize=(11,5))
    # cmin = 1e8
    # cmax = 1e11

    para_slice = para_DA.mean('chi').sel(q=q_slice, energy=e_slice)  # .plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
    perp_slice = perp_DA.mean('chi').sel(q=q_slice, energy=e_slice)  # .plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

    cmin = perp_slice.compute().quantile(0.01)
    cmax = para_slice.compute().quantile(0.995)

    para_slice.plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
    perp_slice.plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
    cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    cbar.set_label(label='Double-Norm-Corrected Intensity [arb. units]', labelpad=12, rotation=270)

    fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
    # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
    # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN & Fluorescence Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)

    fig.set(tight_layout=True)

    axs[0].set(xscale='log', title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
    axs[1].set(xscale='log', title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')

    savePath = plotsPath.joinpath('rsoxs_carbon_reverse/I_maps_v1')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath( 
        f'{sample_name}_{e_slice.start}-{e_slice.stop}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)

    # plt.show()
    plt.close('all')

In [ ]:
# Select polar dataset, to I cuts, individual polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}'].copy()
DS = rsoxs_datasets[f'polar_{edge}_tilted'].copy()

chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

# make selection
# sample_name = 'PM6-Y6_3000_dSiN'
intensity_type = 'corr'
pol = 0

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 

        # Plot
        fig, axs = plt.subplots(1, 2, figsize=(11,5))

        para_slice = para_DA.mean('chi').sel(q=q_slice, energy=e_slice)  # .plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
        perp_slice = perp_DA.mean('chi').sel(q=q_slice, energy=e_slice)  # .plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

        cmin = perp_slice.compute().quantile(0.01)
        cmax = para_slice.compute().quantile(0.995)

        para_slice.plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
        perp_slice.plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

        # Add colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
        cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        cbar.set_label(label='Double-Norm-Corrected Intensity [arb. units]', labelpad=12, rotation=270)

        fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
        # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
        # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN & Fluorescence Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)

        fig.set(tight_layout=True)

        axs[0].set(xscale='log', title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
        axs[1].set(xscale='log', title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')

        savePath = plotsPath.joinpath('rsoxs_carbon_tilted/I_maps_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath( 
            f'{sample_name}_{e_slice.start}-{e_slice.stop}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)

        # plt.show()
        plt.close('all')

In [ ]:
# a = para_EvsQ_avg.hvplot.line(groupby="energy", loglog=True)
# a * perp_EvsQ_avg.hvplot.line(groupby="energy")

### 4. Meaned 1D intensity lineplots, selected energies

In [ ]:
# trmsn90_bkgs_DA = xr.concat(trmsn90_bkgs, dim='sample_name')
# trmsn90_bkgs_DA

In [ ]:
# for sample_name in trmsn90_bkgs_DA.sample_name.values:
#     trmsn90_bkgs_DA.sel(sample_name=sample_name).plot()
#     plt.show()

In [ ]:
# fl_bkgs_DA = xr.concat(fl_bkgs, dim='sample_name')
# fl_bkgs_DA

In [ ]:
# trmsn90_corr.plot()

In [ ]:
# manual_scale_factors_v4 = {
#     'PM6_CB_3000': 8.4,
#     'PM6_CBCN': 11,
#     'PM6_CF_3000': 6,
#     'PM6_CFCN': 10.5,
#     'PM6-Y6_CB': 6.3,
#     'PM6-Y6_CBCN': 4.5,
#     'PM6-Y6_CF': 6,
#     'PM6-Y6_CFCN': 11.5,
#     'PM6-Y6BO_CB': 7.5,
#     'PM6-Y6BO_CF': 5,
#     'PM6-Y6BO_CFCN': 12.8,
#     'PM6-Y7_CB': 6.5,
#     'PM6-Y7_CBCN': 8.5,
#     'PM6-Y7_CF': 4.5,
#     'PM6-Y7_CFCN': 12.8,
#     'PM6-Y7BO_CB': 5,
#     'PM6-Y7BO_CBCN': 8.2,
#     'PM6-Y7BO_CF': 5,
#     'PM6-Y7BO_CFCN': 10,
#     'Y6_CB_2000_1': 5,
#     'Y6_CB_2000': 6,
#     'Y6_CB_2500': 6,
#     'Y6_CB_3000': 6,
#     'Y6_CF_2000': 18,
#     'Y6_CF_2500': 20,
# }

# len(manual_scale_factors_v4)

In [ ]:
# make selection, averaged polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_reverse']

intensity_type = 'corr'
qlims = (0.009, 0.08)
chi_width = 90

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, qlims, chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
#     # Fl correction?
#     scale_factor = float(para_DA.sel(energy=slice(285,305), q=slice(2e-2,6e-2)).mean('chi').mean('energy').integrate('q'))
#     manual_scale_factor = manual_scale_factors_v4[sample_name]
#     trmsn90_corr = (manual_scale_factor * scale_factor * trmsn90_bkgs_DA.sel(sample_name=sample_name))
    
#     para_DA = para_DA - trmsn90_corr
#     perp_DA = perp_DA - trmsn90_corr
        
    # Plot
    energies = para_DA.energy.sel(energy=[260, 275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 338.75], method='nearest').data

    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))

    fig, axs = plt.subplots(ncols=2,figsize=(8,5), tight_layout=True)

    qmin = qlims[0]
    qmax = qlims[1]

    for j, energy in enumerate(energies):
        (para_DA.sel(q=slice(qmin, qmax), energy=energy).mean('chi')
         .plot.line(ax=axs[0], color=colors[j], yscale='log', xscale='log', label=energy))
        (perp_DA.sel(q=slice(qmin, qmax), energy=energy).mean('chi')
         .plot.line(ax=axs[1], color=colors[j], yscale='log', xscale='log', label=energy))

    fig.suptitle(f'IvsQ, {pol}° pol, 90° chi width: {sample_name}', x=0.47)
    # fig.suptitle(f'IvsQ ($Fl$ Corr (C={manual_scale_factor})), {pol}° pol, 90° chi width: {sample_name}', x=0.47)

    axs[0].set(title=f'Parallel to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    axs[0].set_ylim(bottom=1e8)
    axs[1].set(title=f'Perpendicular to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    axs[1].set_ylim(bottom=1e8)
    axs[1].legend(title='Energy [eV]', loc=(1.05,0.1))

    savePath = plotsPath.joinpath('rsoxs_carbon_reverse/I_cuts_v1')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath(
        f'{sample_name}_chiWidth-{chi_width}deg_q{qmin}-{qmax}_pol{pol}deg.png'), dpi=120)

    # plt.show()
    plt.close('all')

In [ ]:
# make selection, individual polarizations
edge = 'carbon'
# Drsoxs_datasets rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'corr'
qlims = (0.009, 0.08)
chi_width = 90

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, qlims, chi_width) 

        # Plot
        energies = para_DA.energy.sel(energy=[260, 275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 338.75], method='nearest').data

        cmap = plt.cm.turbo.copy()
        colors = cmap(np.linspace(0, 1, len(energies)))

        fig, axs = plt.subplots(ncols=2,figsize=(8,5), tight_layout=True)

        qmin = qlims[0]
        qmax = qlims[1]

        for j, energy in enumerate(energies):
            (para_DA.sel(q=slice(qmin, qmax), energy=energy).mean('chi')
             .plot.line(ax=axs[0], color=colors[j], yscale='log', xscale='log', label=energy))
            (perp_DA.sel(q=slice(qmin, qmax), energy=energy).mean('chi')
             .plot.line(ax=axs[1], color=colors[j], yscale='log', xscale='log', label=energy))

        fig.suptitle(f'IvsQ, {pol}° pol, 90° chi width: {sample_name}', x=0.47)
        # fig.suptitle(f'IvsQ ($Fl$ Corr (C={manual_scale_factor})), {pol}° pol, 90° chi width: {sample_name}', x=0.47)

        axs[0].set(title=f'Parallel to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
        axs[0].set_ylim(bottom=1e8)
        axs[1].set(title=f'Perpendicular to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
        axs[1].set_ylim(bottom=1e8)
        axs[1].legend(title='Energy [eV]', loc=(1.05,0.1))

        savePath = plotsPath.joinpath('rsoxs_carbon_tilted/I_cuts_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath(
            f'{sample_name}_chiWidth-{chi_width}deg_q{qmin}-{qmax}_pol{pol}deg.png'), dpi=120)

        # plt.show()
        plt.close('all')

## Anisotropy Ratio Plots

### 1. 2D AR Maps

In [ ]:
DS

In [ ]:
# make selection, averaged polarizations
edge = 'carbon'
# # DS = rsoxs_datasets[f'polar_{edge}']
# DS = rsoxs_datasets[f'polar_{edge}_reverse']

intensity_type = 'raw'
chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

for sample_name in DS.sample_name.values[:]:  
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot
    vlim = 0.4
    ax = ar_DA.sel(energy=e_slice).plot.pcolormesh(figsize=(8,5), norm=plt.Normalize(-vlim, vlim))
    # ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5))

    ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
    ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]', xscale='log')
    ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)

    # savePath = plotsPath.joinpath('rsoxs_carbon_reverse/ar_maps_v1')
    # savePath.mkdir(exist_ok=True)
    # ax.figure.savefig(savePath.joinpath( 
    #     f'vset_{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_energy{e_slice.start}-{e_slice.stop}_pol{pol}deg.png'), 
    #                   dpi=120)

    plt.show()
    plt.close('all')

In [ ]:
%matplotlib inline

In [ ]:
# make selection, averaged polarizations
edge = 'carbon'
# # DS = rsoxs_datasets[f'polar_{edge}']
# DS = rsoxs_datasets[f'polar_{edge}_reverse']

intensity_type = 'raw'
chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

for sample_name in DS.sample_name.values[:]:  
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) #/ (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot
    # vlim = 0.4
    cmap = plt.cm.turbo
    cmap.set_bad('black')
    cmax = ar_DA.sel(energy=e_slice).compute().quantile(0.999)
    ax = ar_DA.sel(energy=e_slice).plot.pcolormesh(figsize=(8,5), norm=LogNorm(1e0, cmax), cmap=cmap)
    # ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5))

    ax.figure.suptitle('$\parallel$ - $\perp$ Intensity Absolute Difference', fontsize=14, x=0.43)
    ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]', xscale='log')
    ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)

    savePath = plotsPath.joinpath('rsoxs_carbon_para-perp_maps_v1')
    savePath.mkdir(exist_ok=True)
    ax.figure.savefig(savePath.joinpath( 
        f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_energy{e_slice.start}-{e_slice.stop}_pol{pol}deg.png'), 
                      dpi=120)

    # plt.show()
    plt.close('all')

In [ ]:
cmin

In [ ]:
cmax

In [ ]:
# make selection, individual polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

for sample_name in tqdm(DS.sample_name.values[:]):  
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 


        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        # Plot
        vlim = 0.5
        ax = ar_DA.sel(energy=e_slice).plot.pcolormesh(figsize=(8,5), norm=plt.Normalize(-vlim, vlim))
        # ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5))

        ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
        ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]', xscale='log')
        ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)

        savePath = plotsPath.joinpath('rsoxs_carbon_tilted/ar_maps_v1')
        savePath.mkdir(exist_ok=True)
        ax.figure.savefig(savePath.joinpath( 
            f'vset_{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_energy{e_slice.start}-{e_slice.stop}_pol{pol}deg.png'), 
                          dpi=120)

        # plt.show()
        plt.close('all')

### 2. 1D meaned AR linecuts

In [ ]:
DS

In [ ]:
# make selection
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
# DS = rsoxs_datasets[f'polar_{edge}_reverse']

intensity_type = 'raw'
chi_width = 90
q_slice = slice(0.008, None)
e_slice = slice(282, 292)

selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
                    'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
                    'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 

for sample_name in DS.sample_name.values[:]:
# for sample_name in tqdm(selected_samples):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot   
    energies = para_DA.energy.sel(energy=[260, 275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 338.75], method='nearest').data
    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))        

    # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
    # cax = ax.inset_axes([1.03, 0, 0.03, 1])
    # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    # cbar.set_label(label=f'Time [seconds]', labelpad=14)
    # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

    fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

    for j, energy in enumerate(energies):
        (ar_DA.sel(energy=energy).plot.line(ax=ax, color=colors[j], label=energy, xscale='log'))

    fig.suptitle(f'Anisotropy Ratio Linecuts: {sample_name}', x=0.46, y=0.95)

    ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(-0.5, 0.5), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
    ax.legend(title='Energy [eV]', loc=(1.03,0.02))

    # savePath = plotsPath.joinpath('rsoxs_carbon_reverse/ar_cuts_v1')
    # savePath.mkdir(exist_ok=True)
    # fig.savefig(savePath.joinpath( 
    #     f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

    plt.show()
    plt.close('all')

In [ ]:
# make selection, individual polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

for sample_name in tqdm(DS.sample_name.values[:]):
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 

        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        # Plot   
        energies = para_DA.energy.sel(energy=[260, 275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 338.75], method='nearest').data
        cmap = plt.cm.turbo.copy()
        colors = cmap(np.linspace(0, 1, len(energies)))        

        # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
        # cax = ax.inset_axes([1.03, 0, 0.03, 1])
        # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        # cbar.set_label(label=f'Time [seconds]', labelpad=14)
        # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

        fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

        for j, energy in enumerate(energies):
            (ar_DA.sel(energy=energy).plot.line(ax=ax, color=colors[j], label=energy, xscale='log'))

        fig.suptitle(f'Anisotropy Ratio Linecuts: {sample_name}', x=0.46, y=0.95)

        # ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(-0.5, 0.5), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
        ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')

        ax.legend(title='Energy [eV]', loc=(1.03,0.02))

        # savePath = plotsPath.joinpath('rsoxs_carbon_tilted/ar_cuts_v1')
        # savePath.mkdir(exist_ok=True)
        # fig.savefig(savePath.joinpath( 
        #     f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

        # plt.show()
        plt.close('all')

### Para - perp log scale intensity

In [ ]:
%matplotlib inline

In [ ]:
plotsPath

In [ ]:
porod_slice

In [ ]:
DS

In [ ]:
DS

In [ ]:
%matplotlib inline

In [ ]:
# make selection
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
# DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'raw'
chi_width = 90
q_slice = slice(0.01, None)
e_slice = slice(282, 292)
porod_slices = []
# porod_slices = [slice(0.0107, 0.0138), # Calculate 1 degree polynomial (straigh line) through these regions
#                 slice(0.0107, 0.0240),
#                 slice(0.0250, 0.0500),
#                 slice(0.0500, 0.0800)]

# selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
#                     'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
#                     'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 
# selected_samples = ['PM6_5CN-CB', 'PM6_CF']

# plt.close('all')

all_slopes = {}
for sample_name in DS.sample_name.values[:]:
# for sample_name in tqdm(selected_samples):
    pol_paras = []
    pol_perps = []
    slopes = {}
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    # ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))
    ar_DA = np.abs(para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot   
    energies = para_DA.energy.sel(energy=[270, 283, 284.4, 284.8, 284.9, 285, 285.1, 285.2, 285.3, 285.4, 286.2, 338.75], method='nearest').data
    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))        

    # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
    # cax = ax.inset_axes([1.03, 0, 0.03, 1])
    # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    # cbar.set_label(label=f'Time [seconds]', labelpad=14)
    # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

    fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

    for j, energy in enumerate(tqdm(energies, desc='Plotting & fitting slopes per energy')):
        sel_DA = ar_DA.sel(energy=energy)
        sel_DA.plot.line(ax=ax, color=colors[j], label=energy, xscale='log', yscale='log',
                         linestyle='None', marker='.', markersize=3)
        
        porod_slopes = {}
        for porod_slice in porod_slices:
            m, b = np.polyfit(np.log(sel_DA.sel(q=porod_slice).q.data), np.log(sel_DA.sel(q=porod_slice).data.compute()), deg=1)
            # display(porod_slice, m)       
            porod_slopes[f'{porod_slice.start}-{porod_slice.stop}'] = np.round(m, 2)
            
        slopes[energy] = porod_slopes
        
    all_slopes[sample_name] = slopes
    fig.suptitle(f'$\parallel$ - $\perp$ Intensity Absolute Difference: {sample_name}', x=0.46, y=0.95)

    ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(1e0, None), ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    # ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(None, None), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
    ax.axes.grid(visible=True, which='both', axis='x')
    legend = ax.legend(title='Energy [eV]', loc=(1.03,0.02))
    for handle in legend.legend_handles:
        handle.set_markersize(15)
    
    savePath = plotsPath.joinpath('rsoxs_carbon_para-perp_cuts_v1')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath( 
        f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

    plt.show()
    plt.close('all')
    
# # Save all_slopes.json in same save folder
# json_data = json.dumps(all_slopes)
# with open(str(savePath.joinpath('all_slopes.json')), 'w') as f:
#     f.write(json_data)

In [ ]:
plotsPath

### Load solution SAXS PM6 data text files

In [ ]:
from scipy.ndimage import gaussian_filter1d

In [ ]:
# Check files
saxsPath = pathlib.Path('/nsls2/data/sst/proposals/2024-2/pass-313412/processed_data/solution_saxs_txts')
# saxsPath = pathlib.Path('/nsls2/data/sst/proposals/2023-3/pass-313412/processed_data/prsoxs_plots/carbon/PM6_solution_saxs_txts')
# saxsPath = outPath.joinpath('prsoxs_plots/carbon/PM6_solution_saxs_txts')
display([f.name for f in sorted(saxsPath.glob('*'))[:8]])
display([f.name for f in sorted(saxsPath.glob('*'))[8:]])

In [ ]:
saxs_arrs = {}

for f in sorted(saxsPath.glob('*')):
    # print('_'.join(f.name.split('_')[:2]))
    var_name = '_'.join(f.name.split('_')[:2])
    saxs_arrs[var_name] = np.loadtxt(f)

saxs_arrs.keys()

In [ ]:
# Load data into dict of numpy arrays
saxs_arrs = {}

for f in sorted(saxsPath.glob('*')):
    # print('_'.join(f.name.split('_')[:2]))
    var_name = '_'.join(f.name.split('_')[:2])
    saxs_arrs[var_name] = np.loadtxt(f)

# saxs_arrs.keys()

# Smooth saxs with gaussian filter:
sigma = 3  # In pixels
for var_name, arr in saxs_arrs.items():
    saxs_arrs[var_name][:,1] = gaussian_filter1d(arr[:,1], sigma)
    
# Put numpy arrays into DataArrays
# saxs_DAs = []
saxs_DS = xr.Dataset(attrs={'name':'PM6_Solution_SAXS'})
for var_name, arr in saxs_arrs.items():
    saxs_DA = xr.DataArray(data=arr[:,1], dims=['q'], coords={'q':arr[:,0], 'err':('q', arr[:,2])})
    saxs_DS[var_name] = saxs_DA

saxs_DS

In [ ]:
# # Ensure q values are equal between datasets, else interpolation needed
# print(np.array_equal(pm6_cb[:,0], pm6_cf[:,0]))
# print(np.array_equal(pm6_cf[:,0], pm6_cn[:,0]))

In [ ]:
# qmin = 0.008
qmin = None
# qmax = float(full_DS.q.max())
qmax = None

plt.close('all')
fig, ax = plt.subplots()
for sample in ['PM6_CB', 'PM6_CF', 'PM6_CN', 'PM6_TO']:
# for sample in saxs_DS.data_vars:
    saxs_DS[sample].sel(q=slice(qmin,qmax)).plot.line(ax=ax, xscale='log',yscale='log', label=sample)

ax.set_ybound(lower=1e-5)
ax.legend()
plt.show()

In [ ]:
# qmin = 0.008
qmin = None
# qmax = float(full_DS.q.max())
qmax = None

plt.close('all')
fig, ax = plt.subplots()
# for sample in ['PM6_CB', 'PM6_CF', 'PM6_CN', 'PM6_TO']:
for sample in ['PM7_CB', 'PM7_CF', 'PM7_CN', 'PM7_TO']:
# for sample in saxs_DS.data_vars:
    saxs_DS[sample].sel(q=slice(qmin,qmax)).plot.line(ax=ax, xscale='log',yscale='log', label=sample)

ax.set_ybound(lower=1e-5)
ax.legend()
plt.show()

In [ ]:
%matplotlib widget

In [ ]:
# qmin = 0.008
qmin = None
# qmax = float(full_DS.q.max())
qmax = None

plt.close('all')
fig, ax = plt.subplots()
for sample in ['Y6_CN', 'Y7_CB', 'Y7_CN']:
# for sample in ['Y7_CB', 'Y7_CN']:
    saxs_DS[sample].sel(q=slice(qmin,qmax)).plot.line(ax=ax, xscale='log',yscale='log', label=sample)

ax.set_ybound(lower=1e-5)
ax.legend()
plt.show()

### Plot para-perp P-RSoXS + solution SAXS overlays

In [ ]:
%matplotlib widget

In [ ]:
sn = {
    'PM6_CF': 'PM6, CF',
    'PM6_5CN-CF': 'PM6, CF 5%CN',
    'PM6_5CN-CB': 'PM6, CB 5%CN'
}

In [ ]:
DS

In [ ]:
%matplotlib widget

In [ ]:
DS.sample_name.values

In [ ]:
# make selection
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']

intensity_type = 'raw'
soln_mat = 'PM7'
chi_width = 90
q_slice = slice(0.01, None)
# e_slice = slice(282, 292)
energy = 285.2
# energy = 284.8
# energies = [260, 283, 284.4, 284.8, 285.2, 286.2, 338.75]
# shift_qval = 0.0104  # value by which to look to shift data, divide rsoxs by this value
shift_qval = 0.02  # value by which to look to shift data, divide rsoxs by this value
# shift_qval = 0.05

# selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
#                     'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
#                     'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 
# selected_samples = ['PM6_CF', 'PM6_5CN-CB']
# selected_samples = ['PM6_5CN-CF']
# selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB']

# selected_samples = ['PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF']
# selected_samples = [f'{soln_mat}_CF', f'{soln_mat}_p5CN-CF', f'{soln_mat}_1CN-CF', f'{soln_mat}_5CN-CF']
# selected_samples = [f'{soln_mat}_p5CN-CB', f'{soln_mat}_1CN-CB', f'{soln_mat}_5CN-CB']
selected_samples = [f'{soln_mat}_CF', f'{soln_mat}_5CN-CB', f'{soln_mat}_TO']

# selected_samples = ['PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB']

# selected_samples = ['Y6_CF', 'Y6_p5CN-CF', 
#                     'Y6_4CF-1CB', 'Y6_2CF-3CB', 'Y6_p5CN-2CF-3CB', 
#                     'Y6_CB', 'Y6_p5CN-CB',
#                     'Y6BO_CF', 'Y6BO_p5CN-CF', 'Y6BO_CB', 'Y6BO_p5CN-CB']
# selected_samples = ['Y6_4CF-1CB', 'Y6_2CF-3CB', 'Y6_p5CN-2CF-3CB']
# selected_samples = ['Y6_CB', 'Y6_p5CN-CB']

# saxs_samples = ['PM6_CF', 'PM6_CB', 'PM6_CN']
# saxs_samples = [f'{soln_mat}_CF', f'{soln_mat}_CB', f'{soln_mat}_CN', f'{soln_mat}_TO']
saxs_samples = [f'{soln_mat}_CF', f'{soln_mat}_CN', f'{soln_mat}_TO']

# saxs_samples = ['PM6_CF', 'PM6_CN']
# saxs_samples = ['Y6_CN', 'Y7_CN', 'Y7_CB']
saxs_sample_to_align = f'{soln_mat}_CN'
selected_qval = float(saxs_DS[saxs_sample_to_align].sel(q=shift_qval, method='nearest'))

# Shift all saxs to line up with selected sample qval
factor1 = selected_qval / float(saxs_DS[f'{soln_mat}_CB'].sel(q=shift_qval, method='nearest'))
factor2 = selected_qval / float(saxs_DS[f'{soln_mat}_CF'].sel(q=shift_qval, method='nearest'))
factor3 = selected_qval / float(saxs_DS[f'{soln_mat}_TO'].sel(q=shift_qval, method='nearest'))
pm6_cb_shifted = saxs_DS[f'{soln_mat}_CB'] * factor1
pm6_cf_shifted = saxs_DS[f'{soln_mat}_CF'] * factor2
pm6_to_shifted = saxs_DS[f'{soln_mat}_TO'] * factor3

# factor1 = selected_qval / float(saxs_DS['Y7_CN'].sel(q=shift_qval, method='nearest'))
# factor2 = selected_qval / float(saxs_DS['Y7_CB'].sel(q=shift_qval, method='nearest'))
# y7_cn_shifted = saxs_DS['Y7_CN'] * factor1
# y7_cb_shifted = saxs_DS['Y7_CB'] * factor2

# saxs_colors = plt.cm.RdPu(np.linspace(0.4,1,len(saxs_samples)))
saxs_colors = plt.cm.Blues(np.linspace(0.5,1,len(saxs_samples)))
plt.close('all')

# for selected_samples in (['Y6_CF', 'Y6_p5CN-CF'],
#                          ['Y6_4CF-1CB', 'Y6_2CF-3CB', 'Y6_p5CN-2CF-3CB'], 
#                          ['Y6_CB', 'Y6_p5CN-CB']):
# for selected_samples in (['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB'],
#                          ['PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF'],
#                          ['PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB']):
rsoxs_colors = plt.cm.Blues(np.linspace(0.3,0.8,len(selected_samples)))
fig, ax = plt.subplots(tight_layout=True, figsize=(4.8,4.3), dpi=130)
for i, sample_name in enumerate(tqdm(selected_samples)):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))
    DA = ar_DA.sel(energy=energy, method='nearest')

    # Shift RSoXS to line up with SAXS
    rsoxs_qval = float(DA.sel(q=shift_qval, method='nearest'))
    rsoxs_shift_factor = selected_qval / rsoxs_qval
    DA = DA * rsoxs_shift_factor

    # Plot rsoxs diff profile:
    DA.plot.line(ax=ax, color=rsoxs_colors[i], label=f'Film RSoXS: {sample_name}', 
                 linestyle='None', marker='.', markersize=3)
    
pm6_cf_shifted.plot.line(ax=ax, color=saxs_colors[0], label=f'Solution SAXS: {soln_mat} in CF')    
# pm6_cb_shifted.plot.line(ax=ax, color=saxs_colors[2], label=f'Solution SAXS: {soln_mat} in CB')
# y7_cn_shifted.plot.line(ax=ax, color=saxs_colors[1], label='Solution: Y7 in CN') 
# y7_cb_shifted.plot.line(ax=ax, color=saxs_colors[0], label='Solution: Y7 in CB') 
saxs_DS[saxs_sample_to_align].plot.line(ax=ax, color=saxs_colors[1], label=f'Solution SAXS: {soln_mat} in CN') 
pm6_to_shifted.plot.line(ax=ax, color=saxs_colors[2], label=f'Solution SAXS: {soln_mat} in TO')


ax.set(xscale='log', yscale='log', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
ax.set(title=f'{soln_mat} film P-RSoXS and solution SAXS overlays \n RSoXS profile: $I_\parallel(Q, E) - I_\perp(Q, E)$, $E={energy}$ eV', ylim=(None, None), xlim=(None, 0.4))
ax.grid(visible=True, which='both', axis='x')
# ax.legend(title='Sample', loc=(1.02,0.2))
AA_str = '$Å^{-1}$'
legend = ax.legend(title=f'Data plotted:')
for handle in legend.legend_handles:
    handle.set_markersize(18)

plt.show()
# plt.close('all')

## Loop for saving many plots

In [ ]:
rsoxs_datasets['polar_oxygen']

In [ ]:
# make selection
edge = 'oxygen'
intensity_type = 'corr'
qlims = (0.01, 0.08)
chi_width = 30
energy_min = 525
energy_max = 545
cmin = 5e8
cmax = 5e10

for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'].sample_name.data):
    for pol in [0, 90]:
        ### Select para & perp DataArrays
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)  
        
        ### ISI:
        # Slice ISI data
        para_ISI = para_DA.interpolate_na(dim='q').mean('chi').sum('q')
        perp_ISI = perp_DA.interpolate_na(dim='q').mean('chi').sum('q')
        
        # Plot
        fig, ax = plt.subplots()
        para_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='para', yscale='log')
        perp_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='perp', yscale='log')
        # para_ISI.plot.line(ax=ax, label='para', yscale='log')
        # perp_ISI.plot.line(ax=ax, label='perp', yscale='log')        
        fig.suptitle('Integrated Scattering Intensity (ISI)', fontsize=14)
        ax.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', xlabel='Photon Energy [eV]', ylabel='Double-Norm-Corrected Intensity [arb. units]')
        ax.legend()
        fig.savefig(plotsPath.joinpath('isi_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')
        
        ### Linecut Maps:
        fig, axs = plt.subplots(1, 2, figsize=(11,5))

        para_DA.mean('chi').sel(energy=slice(energy_min,energy_max)).plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
        perp_DA.mean('chi').sel(energy=slice(energy_min,energy_max)).plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

        sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
        cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        cbar.set_label(label='Intensity [arb. units]', labelpad=12)
        fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
        fig.set(tight_layout=True)
        axs[0].set(title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
        axs[1].set(title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')
        fig.savefig(plotsPath.joinpath('linecut_maps_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')

        ### AR Maps:
        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        if para_DA.sample_name=='BareSiN':
            bareSiN_ar_DA = ar_DA.copy()
            
        # # Subtract AR from SiN:
        # ar_DA = ar_DA - bareSiN_ar_DA

        # Plot
        ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5), norm=plt.Normalize(-0.6, 0.6))
        ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
        ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]')
        ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)
        ax.figure.savefig(plotsPath.joinpath('ar_maps_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')

## Incorporating simulation output
old and incomplete

In [ ]:
import h5py

In [ ]:
whos PosixPath

In [ ]:
simsPath = rootPath.joinpath('imgs_analysis/sim_runs')

In [ ]:
[f.name for f in simPath.iterdir()]

In [ ]:
simPath = simsPath.joinpath('D1.0_a1.0_eps1.0_250pix_375size_0.5m_600steps_1')
h5path = simPath.joinpath('HDF5')
h5list = sorted(h5path.glob('E*h5'))

In [ ]:
def print_key(f, key):
    try:
        keys2 = f[key].keys()
        for key2 in keys2:
            new_key = key + '/' + key2
            print_key(f, new_key)
    except AttributeError:
        print(key)

with h5py.File(h5list[0],'r') as f:
    for key in f.keys():
        print_key(f, key)

#### The scattering pattern is contained within 'K0/projection.' If you simulated multiple incident angles there would be additional 'K#' keys. KIDList contains the vector values for each of the incident angles (propagation direction).

#### Instantiate the PyHyperScattering loader and integrator

In [ ]:
load = phs.load.cyrsoxsLoader(use_chunked_loading=True)
integ = phs.integrate.WPIntegrator(force_np_backend=True) # avoiding gpu backend for this tutorial

In [ ]:
raw = load.loadDirectory(simPath)

In [ ]:
raw

In [ ]:
remeshed = integ.integrateImageStack(raw)

In [ ]:
remeshed

In [ ]:
remeshed.sel(energy=285.1).plot()

In [ ]:
# c = cm.jet(np.linspace(0,1,len(remeshed)))

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,3),dpi=140,constrained_layout=True)
raw.sel(energy=275).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[0],add_colorbar=False)
raw.sel(energy=285).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[1],add_colorbar=False)
raw.sel(energy=295).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[2])

[{axes.set_xlim(-0.1,0.1),axes.set_ylim(-0.1,0.1)} for axes in ax]
plt.show()

In [ ]:
loaded_rsoxs.build_tiff_stack??